# Final Project: Predicting Football Tables Throughout in the Season

## 1. Topic Introduction

### 1a. Why Soccer?

For many years now I have been a fan of soccer, European club soccer and the international game in particular. I follow some other sports, but the so-called *Beautiful Game* garners the lionshare of my attention. Its fluidity, internationality and the high stakes are what draw me to it. Like any other sport in the modern era, soccer generates a massive amount of data. While there is plenty of on-the-pitch data that one could look at, I have chosen to analyze data about the "table" or the standings of a league at a given point in time. 

Soccer determines a league champion differently than most American sports. Rather than a playoff tournament, the champion is determined entirely by regular season performance. The team with the most "points" based on wins, draws and losses at the end of the season will be crowned the league winner. If there are 20 teams in a league for example, each team will play all of the others twice for a total of 38 matches. Winning a match nets three points, drawing nets one and zero points are awarded for a loss. If two teams are tied in points, table position is determined by a statistic called goal differential (GD), which is simply *goals scored - goals against*. In soccer there also exists the concept of relegation, where the bottom two or three teams at the end of the season will be relegated from the league, or dropped down to play at a lower level and replaced with the best teams from that level in the previous season. 

### 1b. What are you analyzing?

I would like to see based on past data how well goal differential and points at earlier stages of the season can predict what position a team will finish in at the end of the season. So, if a team on matchday ten has 15 points and a -2 goal differential, how accurately can I predict their final position? How much will the accuracy of this prediction improve as the season goes on? After the first matchday position and GD are probably not at all predictive, while on matchday 35 when the season is almost finished position is largely set. At what point in the season can current position predict final position within one palce? Within two places? 

This is a **regression task**, because I am trying to predict a variable (end of season position) based upon a number of features including current position, points, goal differential, wins, draws, losses and position last season. While end of season position is not a continuous variable, in the project guide it is stated that an **ordinal** discrete variables can also be the target of a regression task.

## 2. Qualification for Bonus

I am defining my own project problem in a space that interests me. I have not found information about how table position converges over the course of a season, so I will be answering a question that is not easy to answer with a quick google search. I could not find readily available data to solve the problem, so I created my own large dataset by writing two webscrapers (see section 3 for more information) and then several programs to parse and combine them into a CSV file format. This took additional time and effort beyond simply downloading an existing CSV or other data file. While the problem itself is simple, it is novel and the results could be surprising.

**Project Repo Found Here: https://github.com/derek-watson14/3022-Project-dewa5122**

The repo contains the webscrapers written for the project as well as notebooks for each part.

## 3. Data Introduction


### 3a. Data Source and Citation

I was unable to find a readily available dataset that included weekly tables and not just end-of-season tables. I was able to find that information on a German soccer statistics website called *Transfermarkt.com*, but it was displayed in HTML tables such as the one below. That website also contained information about stadium capacity on a year

#### Source Citation
```
Transfermarkt GmbH &amp; Co. KG. (n.d.). League - Table &amp; Results. Transfermarkt. Retrieved November 1, 2022, from https://www.transfermarkt.com/laliga/spieltagtabelle/wettbewerb/{league_code}?saison_id={year}&amp;spieltag={matchday}
```

<img src="./images/sample-table.png" height="600"/>

#### Creating the Dataset

Luckily, the tables were located at easy to parse URLs such as:
```
https://www.transfermarkt.com/laliga/spieltagtabelle/wettbewerb/GB1?saison_id=2017&spieltag=22
```
The league, season and matchday are all included as URL parameters. I could use a web scraping library like `Scrapy` to download the several thousand pages and another library like `BeautifulSoup` to parse the HTML and transform the data into a CSV row for each row of each table. Even the result of the season regarding relagation and Champions league qualification was codified by hex color code and could be translated to more data\*, which I called a team's "Fate".

Leagues will occasionally change the number of teams in the league, the last time such a change was made across Europe's "Big Five" leagues (the top level leagues of Spain, England, Germany, Italy, France) was in 2004. To keep league size consistent across years, I chose 2004 as my first year of analysis. So I downloaded 18 years of data total.

A few columns were added that weren't there in the original dataset, first a boolean column was added to indicate if that row belongs to a "Final Table" or a table from the end of the season. Second another column was added to indicate where the team from that row ended that season, called "EOS Position" (EOS meaning end of season). A final column was added called "Last Season" that indicates how a team finished the year before.

### 3b. Dataset Overview

In this section I will look at the general size of the data and print the schema which lists the data type and a description of each feature of the dataset.

In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
import matplotlib.pyplot as plt

df = pd.read_csv('./data-retrieval/data/table_data.csv')
schema_df = pd.read_csv('./data-retrieval/data/data_schema.csv', index_col='Feature')

#### Size

The dataset contains 17 columns and 65736 rows

In [2]:
rows, columns = df.shape

print(f"Shape: {rows} rows, {columns} columns")

Shape: 65736 rows, 17 columns


#### Features

There are 9 categorical features and 8 numeric features in the dataset. Many of the categorical features, such as matchday or year or position are expressed as numbers, but are actually categorical. This should be clear with the table below.

In [3]:
schema_df['Feature Type'].value_counts()

Categorical    9
Numerical      8
Name: Feature Type, dtype: int64

In [4]:
schema_df

,Feature Description,Data Type,Feature Type
Feature,,,
Country,Country in which matches are played,String,Categorical
League,League in which matches are played,String,Categorical
Year,Starting year of season,Integer,Categorical
Club,Soccer club (team) for this row,String,Categorical
Matchday,Week of season for this row,Integer,Categorical
Position,Club's table position on this matchday,Integer,Categorical
Fate,Result of season in terms of international qua...,String,Categorical
Matches,Number of matches played by this matchday,Integer,Numerical
Wins,Matches won up to this matchday,Integer,Numerical


In [5]:
# Head of dataset
df.head(2)

,Country,League,Year,Club,Matchday,Position,Fate,Matches,Wins,Draws,Losses,Goals For,Goals Against,Goal Differential,Points,Final Table,EOS Position
0,Spain,La Liga,2004,Real Zaragoza,1,1,League Winner,1,1,0,0,3,1,2,3,False,12
1,Spain,La Liga,2004,Atlético de Madrid,1,2,Champions League,1,1,0,0,2,0,2,3,False,11


Because I found many features to be correlated, I decided to make one more that records how many years a team appeared in the dataset. This gives an indication of the financial/